In [1]:
import uproot
import matplotlib.pyplot as plt
import vector
import awkward as ak
import hist
import numpy as np
import itertools
import math


treename="zcounting/tree"
filename_ntuple="/eos/cms/store/group/comm_luminosity/ZCounting/2022/MC/crab/DYJetsToLL_M_50_LO_FlatPU0to70_Run3Winter22.root"
f1=uproot.open(filename_ntuple)
f1.keys()

['countEvents;1',
 'countEvents/unweightedEvents;1',
 'countEvents/weightedEvents;1',
 'pileupMCTemplateMaker;1',
 'pileupMCTemplateMaker/MC_TrueNInt_outOfTimeEarly;1',
 'pileupMCTemplateMaker/MC_TrueNInt_inTime;1',
 'pileupMCTemplateMaker/MC_TrueNInt_outOfTimeLate;1',
 'zcounting;1',
 'zcounting/tree;1']

In [2]:
data=uproot.open(filename_ntuple+":"+treename)

In [3]:
#Track collection 
electron_track = data.arrays(["Track_pt","Track_eta","Track_phi","Track_charge"],"(Track_pt>30)&(abs(Track_eta)<2.5)&(decayMode==11)")

In [4]:
#Electron collection
electron_ele = data.arrays(["Electron_pt","Electron_eta","Electron_phi","Electron_charge"],"(Electron_pt>30)&(abs(Electron_eta)<2.5)&(decayMode==11)")

In [6]:
eventId = data.arrays(["eventNumber"])
nPileup = data.arrays(["nPU"])
nVertex = data.arrays(["nPV"])

In [5]:
import vector
vector.register_awkward()
myEvent = ak.zip({
        "charge": ak.combinations(electron_ele["Electron_charge"],2),
        "pt" : ak.combinations(electron_ele["Electron_pt"],2),
        "eta" :ak.combinations(electron_ele["Electron_eta"],2),
        "phi" : ak.combinations(electron_ele["Electron_phi"],2),
        
    }, with_name="collection of ele")


myEventt = ak.zip({
        "tcharge": ak.combinations(electron_track["Track_charge"],2),
        "tpt" : ak.combinations(electron_track["Track_pt"],2),
        "teta" :ak.combinations(electron_track["Track_eta"],2),
        "tphi" : ak.combinations(electron_track["Track_phi"],2),
    
    }, with_name="collection of trk")


lefts, rights = ak.unzip(myEvent.charge)
tlefts, trights = ak.unzip(myEventt.tcharge)

mask_Electron = lefts*rights == -1
selEvent = ak.flatten(myEvent[mask_Electron])

mask_Electront = tlefts*trights == -1
selEventt = ak.flatten(myEventt[mask_Electront])


selEventF = myEvent[mask_Electron]
selEventtF = myEventt[mask_Electront]
charge = ak.flatten( ak.cartesian([selEventF["charge"],selEventtF["tcharge"]]))
phi = ak.flatten( ak.cartesian([selEventF["phi"],selEventtF["tphi"]]))
eta = ak.flatten( ak.cartesian([selEventF["eta"],selEventtF["teta"]]))
pt = ak.flatten( ak.cartesian([selEventF["pt"],selEventtF["tpt"]]))

print("charge",charge)
print("eta" , eta)
print("phi" , phi)
print("pt" , pt)


charge [((1, -1), (1, -1)), ((1, -1), (-1, 1)), ((1, ... 1), (-1, 1)), ((-1, 1), (1, -1))]
eta [((1.97, 1.47), (1.97, 1.47)), ((0.369, -0.627, ... ((1.23, 0.113), (0.113, 1.23))]
phi [((-0.88, 2.53), (-0.879, 2.53)), ((-2.07, 1.81, ... ((-2.21, 1.27), (1.27, -2.21))]
pt [((39.8, 49.2), (35.6, 51.6)), ((30.8, 60.6), (, ... )), ((38, 39.2), (31.8, 35))]


In [6]:
##function for calculate deltaR

def deltaREval(phi1, phi2, eta1, eta2):
    dPhi = dphi(phi1,phi2)
    return math.sqrt(dPhi**2+ (eta1-eta2)**2)


def dphi(p1,p2):
    res = p1 - p2
    while res > math.pi:
        res -= 2*math.pi
    while res < -math.pi:
        res += 2*math.pi
    return res

##function for calculate mass

electronMass = 0.000510999
mass_lo = 75
mass_hi = 105

##convert pt,eta,phi,charge to list form 

myPt = pt.to_list()
myEta = eta.to_list()
myPhi = phi.to_list()
myCharge = charge.to_list()


##

drList = []
Pt = []
Eta = []
Phi = []
for idx in range(0,len(myEta)):
    DRL = -999
    DRR = -999
    if(myCharge[idx][0][0]*myCharge[idx][1][0]>0):
        DRL = deltaREval(myPhi[idx][0][0],myPhi[idx][1][0],myEta[idx][0][0],myEta[idx][1][0])
        DRR = deltaREval(myPhi[idx][0][1],myPhi[idx][1][1],myEta[idx][0][1],myEta[idx][1][1])
        
        pttr = (myPt[idx][1][0] , myPt[idx][1][1])
        etatr = (myEta[idx][1][0] , myEta[idx][1][1])
        phitr = (myPhi[idx][1][0] , myPhi[idx][1][1])
        
        Pt.append(pttr)
        Eta.append(etatr)
        Phi.append(phitr)
        
    else:
        DRL = deltaREval(myPhi[idx][0][0],myPhi[idx][1][1],myEta[idx][0][0],myEta[idx][1][1])
        DRR = deltaREval(myPhi[idx][0][1],myPhi[idx][1][0],myEta[idx][0][1],myEta[idx][1][0])
        
        pttr = (myPt[idx][1][0] , myPt[idx][1][1])
        etatr = (myEta[idx][1][0] , myEta[idx][1][1])
        phitr = (myPhi[idx][1][0] , myPhi[idx][1][1])
        
        Pt.append(pttr)
        Eta.append(etatr)
        Phi.append(phitr)
        
    tmpList = [1 if (DRL < 0.3) else 0,1 if(DRR<0.3) else 0]
    drList.append(tuple(tmpList))

# vip 
o = ak.Array(drList)
r ,l = ak.unzip(o)
mask_dr = (l+r) > 0




In [7]:
print("deltar",o)
print("ptlen",len(Pt))
print("leno",len(o))


deltar [(1, 1), (1, 1), (1, 1), (1, 1), (0, 1), (1, ... 1), (1, 1), (1, 1), (1, 1), (1, 1)]
ptlen 404958
leno 404958


In [8]:
op = ak.Array(Pt)[mask_dr]
oe = ak.Array(Eta)[mask_dr]
oph = ak.Array(Phi)[mask_dr]


lpt , rpt = ak.unzip(op)
leta , reta = ak.unzip(oe)
lphi , rphi = ak.unzip(oph)

print("lpt",lpt)
print("rpt",rpt)

print("len(rpt)",len(rpt))
print("len(Pt)",len(Pt))


lpt [35.6, 53.4, 36.9, 52, 69.4, 69.4, 35.9, ... 82.6, 39.9, 34.5, 34.3, 41.8, 31.8]
rpt [51.6, 30.2, 35.9, 45.6, 40.6, 55.9, 41.8, ... 65.8, 91.3, 35.5, 36.7, 34, 47.4, 35]
len(rpt) 398643
len(Pt) 404958


In [9]:
electronMass = 0.000510999
mass_lo = 76
mass_hi = 106

n=0
masses=[]
for n in range(len(lpt)):
    
    electron1 = vector.obj(pt=lpt[n], phi=lphi[n], eta=leta[n], mass=electronMass)
    electron2 = vector.obj(pt=rpt[n], phi=rphi[n], eta=reta[n], mass=electronMass)
    masses.append((electron1 + electron2).mass)
    

In [10]:
print(len(masses))
print(masses[0:4])

398643
[87.60820766062707, 85.59865792826344, 114.46631628544341, 97.46024682011914]


In [11]:
r=0
massz=[]
submassz=[]

for r in range(len(masses)):
    if((masses[r]>mass_lo)&(masses[r]<mass_hi)):
        massz.append(masses[r])
        submassz.append(r)


In [12]:
print(submassz[0:14])

[0, 1, 3, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17]


In [13]:
opp = ak.Array(Pt)
no = ak.to_numpy(opp)
nop = ak.to_numpy(op)

i=0
n=[]
for i in range(len(submassz)):
    q = ak.flatten(np.array(np.where(no==nop[submassz[i]])))
    n.append(q)
flat_list = list(itertools.chain.from_iterable(n)) 

In [14]:
m=0
subm=[]
for m in range(len(flat_list)):
    subm.append(o[flat_list[m]])
    
print(len(subm))

303120


In [15]:
tt= np.array(subm)
y=ak.Array(tt)
print(y)

[(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, ... 1), (1, 1), (1, 1), (1, 1), (1, 1)]


In [16]:
rr ,ll = ak.unzip(y)
mask_drf = (ll+rr)==2
llll=y[mask_drf]
print(len(llll))
print(rr)
print(ll)
print(len(rr))

292548
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
303120


In [17]:
print("Npass",len(llll))
print("Nfail+Npass",len(y))
print("electron|track",len(llll)/len(y))

Npass 292548
Nfail+Npass 303120
electron|track 0.9651227236737926
